In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline 
from utils import Dataset

In [2]:
ds =  Dataset()
data = ds.get_data(['job','housing','loan'])

In [3]:
data

,job,housing,loan
0,management,yes,no
1,technician,yes,no
2,entrepreneur,yes,yes
3,blue-collar,yes,no
4,unknown,no,no
...,...,...,...
45206,technician,no,no
45207,retired,no,no
45208,retired,no,no
45209,blue-collar,no,no


In [4]:
# Helper para acrescentar coluna had_loan
def set_had_loan(row):
    return 'yes' if (row['housing']=='yes' or row['loan']=='yes') else 'no'  

In [5]:
# Acresenta coluna q indica se tem emprestimo
had_loan = data.apply(set_had_loan , axis=1)
data = pd.concat([data,had_loan], axis=1)
data.columns.values[3]='had_loan'

In [6]:
# Conta registros nas colunas 'housing', 'loan', 'had_loan', agrupando por job
count_data = data.groupby('job')['housing', 'loan', 'had_loan'].apply(lambda x: x[x=='yes'].count())

# Conta registros de cada job (utilizando a coluna housing) 
data_tt = pd.DataFrame(data=data.groupby(['job']).count()['housing']) # Contando somente as prof na col housing
data_tt.columns = ['tt']

data_tt = pd.concat([data_tt, count_data], axis=1)

data_tt['perc_housing']=data_tt.housing/data_tt.tt
data_tt['perc_loan']=data_tt.loan/data_tt.tt
data_tt['perc_tt']=data_tt.had_loan/data_tt.tt

In [8]:
data

,job,housing,loan,had_loan
0,management,yes,no,yes
1,technician,yes,no,yes
2,entrepreneur,yes,yes,yes
3,blue-collar,yes,no,yes
4,unknown,no,no,no
...,...,...,...,...
45206,technician,no,no,no
45207,retired,no,no,no
45208,retired,no,no,no
45209,blue-collar,no,no,no


In [10]:
data_tt

,tt,housing,loan,had_loan,perc_housing,perc_loan,perc_tt
job,,,,,,,
admin.,5171,3182,991,3569,0.615355,0.191646,0.690195
blue-collar,9732,7048,1684,7599,0.724209,0.173037,0.780826
entrepreneur,1487,869,356,1016,0.584398,0.239408,0.683255
housemaid,1240,398,152,490,0.320968,0.122581,0.395161
management,9458,4678,1253,5225,0.494608,0.132480,0.552442
...,...,...,...,...,...,...,...
services,4154,2766,836,3063,0.665864,0.201252,0.737362
student,938,249,12,257,0.265458,0.012793,0.273987
technician,7597,4115,1309,4608,0.541661,0.172305,0.606555


In [11]:
profissao_maior_tendencia = data_tt['perc_tt'].idxmax()
registro_maior_tendencia = data_tt.loc[profissao_maior_tendencia]
percent_maior_tendencia = max(registro_maior_tendencia.loan, registro_maior_tendencia.housing)
tipo_seguro_maior_tendencia = 'Loan' if registro_maior_tendencia.loan > registro_maior_tendencia.housing else 'Housing'

In [12]:
print('Profissão de maior tendencia a fazer um empréstimo:', profissao_maior_tendencia)
print('Tipo de seguro mais frequente na prof de maior tendencia:', tipo_seguro_maior_tendencia)

Profissão de maior tendencia a fazer um empréstimo: blue-collar
Tipo de seguro mais frequente na prof de maior tendencia: Housing


In [ ]:
# Dados para a segunda questão
data_q2 =  data[['campaign','previous','poutcome', 'y']]

# Inserir coluna para totalizar os contatos realizados
tt_contacts = data_q2.campaign+data_q2['previous'] 
data_q2.insert(loc=2, column='tt_contacts', value=tt_contacts)

In [ ]:
data_q2

In [ ]:
freq_campaign = data_q2.campaign.value_counts(sort=True)
freq_previous = data_q2.previous.value_counts(sort=True)
freq_tt = data_q2.tt_contacts.value_counts(sort=True)

freq_campaign_relative = data_q2.campaign.value_counts(sort=True, normalize=True)
freq_previous_relative = data_q2.previous.value_counts(sort=True, normalize=True)
freq_tt_relative = data_q2.tt_contacts.value_counts(sort=True, normalize=True)

In [ ]:
freq_tt_relative

In [ ]:
# Min and Max continuous variables:
min_contacts = data_q2.tt_contacts.min()
max_contacts = data_q2.tt_contacts.max()
media_aritimetica = (min_contacts+max_contacts)/2

# Criar var categorica a partir do total dos contatos
qs=[.2,.4,.6,.8,1]
labels=['1','2','3','4']
tt_contacts_categ = pd.qcut(data_q2['tt_contacts'],qs,labels=labels)
data_q2.insert(loc=3,column='tt_contacts_categ', value=tt_contacts_categ)

In [ ]:
data_q2